In [2]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [3]:
#!/usr/bin/env python3
# -- coding: utf-8 --

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import tqdm
import cv2
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog
import time


In [4]:

class Detector:
    def __init__(self, detect_type):
        self.cfg = get_cfg()
        #load model config
        if detect_type == "OD":
          self.cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
          self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
        elif detect_type == "KP":
          self.cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
          self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml")

        self.cfg.MODEL.ROI_HEADS.SCORE_TRESH_TEST = 0.7
        self.cfg.MODEL.DEVICE = 'cuda'  #or "cpu"
        self.cfg.TEST.DETECTIONS_PER_IMAGE = 2
        self.predictor = DefaultPredictor(self.cfg)

    def image_pred(self, path):
        im = cv2.imread(path)
        predictions = self.predictor(im)
        viz = Visualizer(im[:, :, ::-1], metadata= MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), instance_mose= ColorMode.IMAGE_BW)

        out = viz.draw_instance_predictions(predictions["instances"].to('cpu'))

        cv2.imshow('out', out.get_image()[:, :, ::-1])
        cv2.waitKey(0)
    

    def find_girl_bb(self, boxes, frame):
      
      hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
      num_of_bb = len(boxes)
      res = 0
      best_fit = 0
      yellow_hsv = ()
      # Range for yellow
      yellow_lower = np.array([20, 100, 100])
      yellow_upper = np.array([30, 255, 255])

      for indx, box in enumerate(boxes):

        img = hsv[int(box[1]):int(box[3]) , int(box[0]):int(box[2])] 
        mask_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper)
        yellow_output = cv2.bitwise_and(frame, frame, mask=mask_yellow)
        yellow_ratio =(cv2.countNonZero(mask_yellow))/(img.size/3)
        if yellow_ratio > best_fit:
          res = indx
          best_fit = yellow_ratio
      
      return res # returns the line of the box bounding the lady.


    def video_pred(self, path):
      # Extract video properties
      video = cv2.VideoCapture(path)
      width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
      height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
      frames_per_second = video.get(cv2.CAP_PROP_FPS)
      num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

      # Initialize video writer
      video_writer = cv2.VideoWriter('out.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)
      frames = 0
      while True:
        print(frames)
        hasFrame, frame = video.read()
        if not hasFrame:
          print("no more frames")
          break
        frames += 1
        predictions = self.predictor(frame)
        viz = Visualizer(frame[:,:,::-1], metadata= MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), instance_mode= ColorMode.IMAGE)
        
       
        #predictions = predictions[predictions.scores > 0.85]
        target_fields = predictions["instances"].get_fields()

        box_num = self.find_girl_bb(target_fields["pred_boxes"], frame)
        colors = ['g' for x in target_fields["pred_boxes"] ]
        colors[box_num] = 'm'
        names = []
        for x in colors:
          if x == 'm':
            names.append("Emma")
          else:
            names.append("Ryan")
        print(colors)
        out = viz.overlay_instances(boxes= target_fields['pred_boxes'].to('cpu'), labels=names, keypoints=target_fields['pred_keypoints'].to('cpu'), assigned_colors=colors)
        #out = viz.draw_instance_predictions(predictions["instances"].to("cpu"))
        video_writer.write(out.get_image()[:, :, ::-1])
        print(str(frames*100//num_frames) + "% finished")
      video.release()
      video_writer.release()
      cv2.destroyAllWindows()

    
      



        




In [5]:
detector = Detector("KP")
detector.video_pred('/content/in.mp4')

model_final_997cc7.pkl: 313MB [00:06, 45.3MB/s]                           


0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.7/dist-packages/detectron2/structures/keypoints.py:224: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pyt

Streaming output truncated to the last 5000 lines.
904
['g', 'm']
35% finished
905
['g', 'm']
35% finished
906
['g', 'm']
35% finished
907
['g', 'm']
35% finished
908
['g', 'm']
35% finished
909
['m', 'g']
35% finished
910
['g', 'm']
35% finished
911
['g', 'm']
35% finished
912
['m', 'g']
35% finished
913
['m', 'g']
35% finished
914
['m', 'g']
35% finished
915
['m', 'g']
35% finished
916
['m', 'g']
35% finished
917
['g', 'm']
35% finished
918
['m', 'g']
35% finished
919
['m', 'g']
35% finished
920
['g', 'm']
35% finished
921
['g', 'm']
35% finished
922
['m', 'g']
35% finished
923
['m', 'g']
35% finished
924
['m', 'g']
35% finished
925
['g', 'm']
36% finished
926
['g', 'm']
36% finished
927
['g', 'm']
36% finished
928
['g', 'm']
36% finished
929
['g', 'm']
36% finished
930
['g', 'm']
36% finished
931
['g', 'm']
36% finished
932
['g', 'm']
36% finished
933
['g', 'm']
36% finished
934
['g', 'm']
36% finished
935
['g', 'm']
36% finished
936
['g', 'm']
36% finished
937
['g', 'm']
36% finish